### Используемые библиотеки

In [ ]:
import pywt
from enum import Enum
import cv2
from PIL import Image, ImageChops
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Tuple, Dict, Any

In [ ]:
class ColorSpaceType(Enum):
    RGB = 'RGB'
    YCrCb = 'YCrCb'

class SubbandType(Enum):
    LL = "LL"
    LH = "LH"
    HL = "HL"
    HH = "HH"

def get_image_channels(image_path: str, color_space: ColorSpaceType):
    if color_space is ColorSpaceType.RGB:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)

        return {
            'color_space': color_space,
            'channels': img_array
        }
    
    elif color_space is ColorSpaceType.YCrCb:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        img_ycbcr = cv2.cvtColor(img_array, cv2.COLOR_RGB2YCrCb)

        return {
            'color_space': color_space,
            'channels': img_ycbcr
        }
    
    else:
        raise ValueError('Invalid color space.')
        
def save_image_channels(img_data: np.ndarray, out_path: str, color_space: ColorSpaceType):
    if color_space is ColorSpaceType.RGB:
        Image.fromarray(img_data).save(out_path)
        return
    
    elif color_space is ColorSpaceType.YCrCb:
        cbcr_image_data = cv2.cvtColor(img_data, cv2.COLOR_YCrCb2RGB)
        Image.fromarray(cbcr_image_data).save(out_path)
        return
    
    else:
        raise ValueError('Invalid color space.')

## Функции анализа изображений

In [ ]:
# def visualize_diff(img1_path: str, img2_path: str, out_path: str):
#     img1 = Image.open(img1_path).convert('RGB')
#     img2 = Image.open(img2_path).convert('RGB')

#     # Побайтовая разница
#     diff = ImageChops.difference(img1, img2)

#     # Усиливаем разницу (если изменения маленькие)
#     diff = diff.point(lambda p: min(255, p * 10))

#     diff.save(out_path)
#     return diff

# def visualize_mask(img1_path: str, img2_path: str, out_path: str):
#     arr1 = np.array(Image.open(img1_path).convert('RGB'))
#     arr2 = np.array(Image.open(img2_path).convert('RGB'))

#     mask = np.any(arr1 != arr2, axis=-1)

#     heatmap = np.zeros_like(arr1)
#     heatmap[mask] = [255, 0, 0]

#     Image.fromarray(heatmap).save(out_path)
#     return heatmap

# def calculate_psnr_detailed(img1, img2):
#     """
#     Расчет PSNR с детализацией по каналам для цветных изображений
    
#     Возвращает словарь с PSNR для каждого канала и среднее значение
#     """
#     if img1.shape != img2.shape:
#         raise ValueError("Размеры изображений не совпадают")
    
#     if len(img1.shape) == 2:  # Черно-белое изображение
#         mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
#         psnr = 10 * np.log10((255.0 ** 2) / mse) if mse > 0 else float('inf')
#         return {"psnr": psnr, "mse": mse}
    
#     elif len(img1.shape) == 3:  # Цветное изображение
#         results = {}
#         channel_names = ['R', 'G', 'B'] if img1.shape[2] == 3 else ['C1', 'C2', 'C3']
        
#         # Вычисляем для каждого канала отдельно
#         for i in range(img1.shape[2]):
#             channel1 = img1[:, :, i]
#             channel2 = img2[:, :, i]
#             mse = np.mean((channel1.astype(float) - channel2.astype(float)) ** 2)
#             psnr = 10 * np.log10((255.0 ** 2) / mse) if mse > 0 else float('inf')
#             results[f"psnr_{channel_names[i]}"] = psnr
#             results[f"mse_{channel_names[i]}"] = mse
        
#         # Среднее по всем каналам
#         all_psnr = [results[f"psnr_{ch}"] for ch in channel_names]
#         results["psnr_avg"] = np.mean(all_psnr)
        
#         all_mse = [results[f"mse_{ch}"] for ch in channel_names]
#         results["mse_avg"] = np.mean(all_mse)
        
#         return results
    
#     else:
#         raise ValueError("Неподдерживаемая размерность изображения")
    
# def calculate_psnr(img1, img2):
#     """
#     Простой расчет PSNR между двумя изображениями формата uint8 [0, 255]
    
#     Параметры:
#     img1, img2 -- numpy массивы одинаковой формы (H, W) или (H, W, C)
    
#     Возвращает:
#     PSNR в децибелах (dB)
    
#     Пример:
#     >>> psnr = calculate_psnr(original_img, stego_img)
#     >>> print(f"Качество: {psnr:.2f} dB")
#     """
    
#     # Проверяем, что изображения одинакового размера
#     if img1.shape != img2.shape:
#         raise ValueError(f"Размеры не совпадают: {img1.shape} != {img2.shape}")
    
#     # Проверяем тип данных
#     if img1.dtype != np.uint8 or img2.dtype != np.uint8:
#         raise ValueError("Изображения должны быть в формате uint8 [0, 255]")
    
#     # Вычисляем MSE (среднеквадратичную ошибку)
#     mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    
#     # Если ошибки нет, PSNR бесконечен
#     if mse == 0:
#         return float('inf')
    
#     # Максимальное значение пикселя для 8-битного изображения
#     MAX_PIXEL = 255.0
    
#     # Вычисляем PSNR
#     psnr = 10 * np.log10((MAX_PIXEL ** 2) / mse)
    
#     return psnr

In [ ]:
def embed_text(image_path: str, out_path: str, text: str, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0) -> dict:
    # 1. Получаем выбранный канал изображения
    img_data = get_image_channels(image_path, color_space)
    channel_data = img_data['channels'][:, :, channel].astype(np.float64)

    # 2. Применяем к каналу DWT (используем symmetric для более стабильного поведения)
    wt_type = 'haar'
    wt_mode = 'symmetric'
    
    LL, (LH, HL, HH) = pywt.dwt2(channel_data, wt_type, mode=wt_mode)
    coeffs_dict = {
        'LL': LL,
        'LH': LH,
        'HL': HL,
        'HH': HH
    }

    # 3. Берем выбранную подполосу и приводим ее к одномерному массиву дл удобства
    selected_subband: np.ndarray[np.float64] = coeffs_dict[target_subband]
    subband = selected_subband.flatten()

    # 4. Модифициурем контейнер (одномерный массив)
    #

    # 5. Выполняем обратные преобразования для получения модифицированного канала
    modied_subband = subband.reshape(selected_subband.shape)
    coeffs_dict[target_subband] = modied_subband
    modified_coeffs = (coeffs_dict['LL'], 
                           (coeffs_dict['LH'], 
                            coeffs_dict['HL'], 
                            coeffs_dict['HH']))
    modified_channel_data = pywt.idwt2(modified_coeffs, wt_type, mode=wt_mode)

    # 6. Сохраняем модифицированный канал в изображении
    img_data['channels'][:, :, channel] = modified_channel_data
    save_image_channels(img_data['channels'], out_path, color_space)

    return {
        'image_data': img_data,
        'selected_channel': channel,
        'channel_data': channel_data,
        'target_subband': target_subband
    }

In [ ]:
def extract_text(image_path: str, color_space: ColorSpaceType = ColorSpaceType.RGB, target_subband: str = 'LL', channel: int = 0) -> dict:
    img_data = get_image_channels(image_path, color_space)
    channel_data = img_data['channels'][:, :, channel].astype(np.float64)

    return {
        'image_data': img_data,
        'selected_channel': channel,
        'channel_data': channel_data,
        'target_subband': target_subband,
    }

In [ ]:
# used_img_path = 'img/marsik-150x200.png'
# embedded_img_path = 'embedded/marsik-150x200-embedded.png'

used_img_path = 'img/marsik-960x1280.png'
embedded_img_path = 'embedded/marsik-960x1280-embedded.png'

secret_string = "Silence in an era of noise: How to find yourself in a world that never stops Our world is immersed in a continuous, intrusive hum. This is not just the physical noise of megacities, but a fundamental information and social backdrop that has become our new habitat. We wake up to the vibration of our smartphones, scroll through our news feeds at breakfast, and are immersed in a whirlwind of notifications, messaging apps, and endless online meetings at work. In the evening, as we try to relax, we unconsciously scroll through social media, consuming the carefully curated lives of others. This permanent digital noise creates the illusion of hyper-connection, but paradoxically leads to deep loneliness and distraction. We know more about the world, but less about ourselves. We have thousands of \"friends,\" but sometimes we don't have anyone to share our true sadness or joy with."

In [ ]:
embedded_result = embed_text(
    used_img_path,
    embedded_img_path,
    secret_string,
    color_space=ColorSpaceType.YCrCb,
    target_subband='HL',
    channel=0
)

# extract_result = extract_text(
#     embedded_img_path,
#     color_space=embedded_result['image_data']['color_space'],
#     target_subband=embedded_result['target_subband'],
#     channel=embedded_result['selected_channel']
# )

In [ ]:
# ============================
# Вспомогательные функции
# ============================

def load_image(path: str) -> np.ndarray:
    """Загружает изображение и возвращает как NumPy массив RGB."""
    img = Image.open(path).convert("RGB")
    return np.array(img)


def save_image_from_array(arr: np.ndarray, path: str) -> None:
    """Сохраняет NumPy-массив RGB в файл."""
    Image.fromarray(arr).save(path)


# ============================
# Разница и маски
# ============================

def image_diff(img1: np.ndarray, img2: np.ndarray, amplify: int = 10) -> np.ndarray:
    """
    Создаёт diff двух изображений с усилением различий.

    amplify — множитель, чтобы лучше выделять слабые различия.
    """
    pil1 = Image.fromarray(img1)
    pil2 = Image.fromarray(img2)
    diff = ImageChops.difference(pil1, pil2)

    # Усиливаем различия
    diff = diff.point(lambda p: min(255, p * amplify))

    return np.array(diff)


def image_mask(img1: np.ndarray, img2: np.ndarray) -> np.ndarray:
    """
    Создаёт бинарную маску различий (красные пиксели — есть разница).
    """
    mask = np.any(img1 != img2, axis=-1)

    heatmap = np.zeros_like(img1)
    heatmap[mask] = [255, 0, 0]  # выделяем красным

    return heatmap


# ============================
# PSNR
# ============================

def calculate_psnr(img1: np.ndarray, img2: np.ndarray) -> float:
    """
    Простой PSNR для RGB или grayscale.
    """
    if img1.shape != img2.shape:
        raise ValueError(f"Размеры не совпадают: {img1.shape} != {img2.shape}")

    mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    if mse == 0:
        return float('inf')

    return 10 * np.log10((255.0 ** 2) / mse)


def calculate_psnr_detailed(img1: np.ndarray, img2: np.ndarray) -> Dict[str, Any]:
    """
    PSNR по каналам + средний PSNR.
    """
    if img1.shape != img2.shape:
        raise ValueError("Размеры изображений не совпадают")

    if img1.ndim == 2:
        # 1 канал
        mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
        psnr = float('inf') if mse == 0 else 10 * np.log10((255.0 ** 2) / mse)
        return {"psnr": psnr, "mse": mse}

    results = {}
    names = ["R", "G", "B"]

    psnrs = []
    mses = []

    for i, ch in enumerate(names):
        c1 = img1[:, :, i].astype(float)
        c2 = img2[:, :, i].astype(float)

        mse = np.mean((c1 - c2) ** 2)
        psnr = float('inf') if mse == 0 else 10 * np.log10((255.0 ** 2) / mse)

        results[f"psnr_{ch}"] = psnr
        results[f"mse_{ch}"] = mse

        psnrs.append(psnr)
        mses.append(mse)

    results["psnr_avg"] = np.mean(psnrs)
    results["mse_avg"] = np.mean(mses)

    return results


# ============================
# Визуальный отчёт
# ============================

def visualize_comparison(
        original: np.ndarray,
        modified: np.ndarray,
        diff: np.ndarray,
        mask: np.ndarray,
        figsize: Tuple[int, int] = (16, 8)
):
    """
    Рисует 4 изображения: исходное, модифицированное, diff и маску различий.
    """
    plt.figure(figsize=figsize)

    plt.subplot(1, 4, 1)
    plt.title("Original")
    plt.imshow(original)
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.title("Modified")
    plt.imshow(modified)
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.title("Diff (amplified)")
    plt.imshow(diff)
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.title("Difference Mask")
    plt.imshow(mask)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


# ============================
# Полный анализ
# ============================

def analyze_images(path1: str, path2: str, amplify: int = 10) -> Dict[str, Any]:
    """
    Полный анализ двух изображений:
    - diff
    - маска различий
    - PSNR
    - PSNR по каналам

    Возвращает словарь с результатами.
    """
    img1 = load_image(path1)
    img2 = load_image(path2)

    diff = image_diff(img1, img2, amplify)
    mask = image_mask(img1, img2)

    psnr = calculate_psnr(img1, img2)
    psnr_detailed = calculate_psnr_detailed(img1, img2)

    visualize_comparison(img1, img2, diff, mask)

    return {
        "psnr": psnr,
        "psnr_detailed": psnr_detailed,
        "diff": diff,
        "mask": mask
    }

In [ ]:
result = analyze_images(
    used_img_path,
    embedded_img_path,
    amplify=15
)

print("PSNR:", result["psnr"])
print(result["psnr_detailed"])

In [ ]:
# diff_path = embedded_img_path.replace('.png', '-diff.png')
# mask_path = embedded_img_path.replace('.png', '-mask.png')

# visualize_diff(used_img_path, embedded_img_path, diff_path)
# mask = visualize_mask(used_img_path, embedded_img_path, mask_path)

In [ ]:
# def test_with_pil():
#     # Загружаем изображения
#     original = Image.open(used_img_path).convert('RGB')
#     stego = Image.open(embedded_img_path).convert('RGB')
    
#     # Конвертируем в numpy массивы
#     original_np = np.array(original)
#     stego_np = np.array(stego)
    
#     # Считаем PSNR
#     psnr = calculate_psnr(original_np, stego_np)
#     print(f"PSNR: {psnr:.2f} dB")
    
#     # Интерпретация результата
#     if psnr == float('inf'):
#         print("Изображения идентичны")
#     elif psnr > 40:
#         print("Отличное качество - изменения не видны")
#     elif psnr > 30:
#         print("Хорошее качество - изменения почти не видны")
#     elif psnr > 20:
#         print("Удовлетворительное качество - изменения заметны")
#     else:
#         print("Плохое качество - сильные искажения")

In [ ]:
# test_with_pil()